In [1]:
%run ../Python_files/util_data_storage_and_load.py

In [2]:
%run ../Python_files/load_dicts.py

In [3]:
%run ../Python_files/util.py

In [4]:
# load logit_route_choice_probability_matrix
P = zload('../temp_files/logit_route_choice_probability_matrix.pkz')

In [5]:
import numpy as np
from numpy.linalg import inv

P = np.matrix(P)

In [6]:
np.size(P, 0), np.size(P, 1)

(56, 140)

In [7]:
import numpy as np
from numpy.linalg import inv

def samp_cov(x):
    """
    x: sample matrix, each column is a link flow vector sample; 24 * K
    K: number of samples
    S: sample covariance matrix
    ----------------
    return: inv(S)
    ----------------
    """
    x = np.matrix(x)
    K = np.size(x, 1)
    x_mean = sum(x[:,k] for k in range(K)) / K
    S = sum(np.dot(x[:,k] - x_mean, np.transpose(x[:,k] - x_mean)) for k in range(K)) / (K - 1)
    return S

In [8]:
# load path-link incidence matrix
A = zload('../temp_files/path-link_incidence_matrix.pkz')

In [9]:
# load link counts data

import json

with open('../temp_files/link_day_minute_Jan_dict_JSON.json', 'r') as json_file:
    link_day_minute_Jan_dict_JSON = json.load(json_file)

In [18]:
#week_day_Jan_list = [2, 3, 4, 5, 6, 9, 10, 11, 12, 13, 16, 17, 18, 19, 20, 23, 24, 25, 26, 27, 30, 31]
week_day_Jan_list = [2, 9, 16, 23, 30]

In [19]:
link_day_minute_Jan_list = []
for link_idx in range(24):
    for day in week_day_Jan_list: 
        for minute_idx in range(120):
            key = 'link_' + str(link_idx) + '_' + str(day)
            link_day_minute_Jan_list.append(link_day_minute_Jan_dict_JSON[key] ['NT_flow_minute'][minute_idx])

In [20]:
len(link_day_minute_Jan_list)

14400

In [21]:
x = np.matrix(link_day_minute_Jan_list)
x = np.matrix.reshape(x, 24, 600)

x = np.nan_to_num(x)
y = np.array(np.transpose(x))
y = y[np.all(y != 0, axis=1)]
x = np.transpose(y)
x = np.matrix(x)

In [22]:
np.size(x, 0), np.size(x, 1)

(24, 600)

In [23]:
x[:,:2]

matrix([[ 3566.37965311,  3530.21548515],
        [ 3408.15693735,  3471.50264591],
        [ 4446.26913018,  4438.78882957],
        [ 4489.37849326,  4447.13222061],
        [ 4463.92996807,  4367.32820001],
        [ 4005.18179424,  4090.18604852],
        [ 3562.77825539,  3625.5069303 ],
        [ 3615.36697367,  3665.77865869],
        [ 4755.94434938,  4888.65619982],
        [ 4785.12614786,  4628.22498787],
        [ 4339.41335095,  4406.70248455],
        [ 3733.35563388,  3694.76111735],
        [ 3387.95601529,  3299.17532231],
        [ 3660.15083428,  3568.87728428],
        [ 3476.35158844,  3655.07470176],
        [ 2937.68010824,  2859.87350561],
        [ 3657.04426948,  3596.84314113],
        [ 3655.16096142,  3670.27365722],
        [ 3941.98220017,  3982.94914084],
        [ 3978.55943801,  4029.57479363],
        [ 3975.35982887,  4001.32179676],
        [ 3902.46066368,  3842.8010075 ],
        [ 3610.46230744,  3509.20812603],
        [ 3682.88323075,  3586.749

In [24]:
np.size(A,0), np.size(A,1)

(24, 140)

In [25]:
from gurobipy import *

L = 56  # dimension of lam

K = np.size(x, 1)
S = samp_cov(x)
inv_S = inv(S)

A_t = np.transpose(A)
P_t = np.transpose(P)
# PA'
PA_t = np.dot(P, A_t)
# AP_t
AP_t = np.transpose(PA_t)

Q = np.dot(np.dot(PA_t, inv_S), AP_t)
b = sum([np.dot(np.dot(PA_t, inv_S), x[:, k]) for k in range(K)])


model = Model("OD_matrix_estimation")

lam = []
for l in range(L):
    lam.append(model.addVar(name='lam_' + str(l)))

model.update() 

# Set objective: (K/2) lam' * Q * lam - b * lam
obj = 0
for i in range(L):
    for j in range(L):
        obj += (1.0 /2) * K * lam[i] * Q[i, j] * lam[j]
for l in range(L):
    obj += - b[l] * lam[l]
model.setObjective(obj)

# Add constraint: lam >= 0
for l in range(L):
    model.addConstr(lam[l] >= 0)

model.update() 

model.optimize()

lam_list = []
for v in model.getVars():
    print('%s %g' % (v.varName, v.x))
    lam_list.append(v.x)
print('Obj: %g' % obj.getValue())

Optimize a model with 56 rows, 56 columns and 56 nonzeros
Model has 1596 quadratic objective terms
Coefficient statistics:
  Matrix range    [1e+00, 1e+00]
  Objective range [6e+00, 1e+03]
  Bounds range    [0e+00, 0e+00]
  RHS range       [0e+00, 0e+00]
Presolve removed 56 rows and 0 columns
Presolve time: 0.00s
Presolved: 0 rows, 56 columns, 0 nonzeros
Presolved model has 1596 quadratic objective terms
Ordering time: 0.00s

Barrier statistics:
 Free vars  : 25
 AA' NZ     : 3.000e+02
 Factor NZ  : 3.250e+02
 Factor Ops : 5.525e+03 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0  -2.35650068e+07  0.00000000e+00  5.84e+03 1.85e+03  1.00e+06     0s
   1  -1.34439937e+07 -6.52882093e+06  1.38e+03 4.37e+02  2.64e+05     0s
   2  -9.53779562e+06 -1.05455762e+07  1.45e+02 4.59e+01  6.01e+04     0s
   3  -9.32641992e+06 -9.72436025e+06  1.45e-04 4.59e-05  7.1

In [26]:
# write estimation result to file
n = 8  # number of nodes
with open('../temp_files/OD_demand_matrix_Jan_weekday_NT.txt', 'w') as the_file:
    idx = 0
    for i in range(n + 1)[1:]:
        for j in range(n + 1)[1:]:
            if i != j: 
                the_file.write("%d,%d,%f\n" %(i, j, lam_list[idx]))
                idx += 1

In [27]:
lam_list

[3.359994475879499e-11,
 7003.158149690515,
 3.748954350466009e-11,
 0.041601646588084684,
 4.744382268536029,
 1.1121358649065236,
 1.1313181148285623e-08,
 0.0005199558218639128,
 1.1571135042771754e-10,
 4423.554469882811,
 1.157113501215416e-10,
 1.1571135042673237e-10,
 1.1571135042067575e-10,
 1.1637489177975162e-10,
 4148.48691196467,
 0.41317120544247565,
 0.23518653331230688,
 94.57039295984511,
 886.8243899829932,
 6236.322719685787,
 0.540002000618216,
 2196.4326470035753,
 1901.8900127878874,
 0.7810444746467624,
 2.2796183925197537,
 3173.0356883039285,
 0.0012035991516392078,
 3477.6934376686277,
 0.20698787336232272,
 0.16619739641670606,
 1188.9497674971979,
 3487.0654314612884,
 343.52091997502237,
 0.0011941098945638176,
 0.001195242661189047,
 696.4859409898917,
 1.756729332901429,
 73.75228751012781,
 0.20554122993264468,
 2704.6047917947944,
 455.8759678248608,
 575.2143157211685,
 383.71975333403395,
 0.6617750758601133,
 2540.554467898457,
 0.1630556150756699,
 3